In [1]:
import re
import os
from langchain.vectorstores.neo4j_vector import Neo4jVector
from langchain.document_loaders import WikipediaLoader
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.text_splitter import (
    CharacterTextSplitter,
    RecursiveCharacterTextSplitter,
)
from langchain.graphs import Neo4jGraph
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from langchain.text_splitter import TokenTextSplitter

from secret import *
import chromadb

os.environ["OPENAI_API_KEY"] = OPENAI_API_KEY

In [2]:
graph = Neo4jGraph(url="bolt://localhost:7687", username="neo4j", password="test1234")
graph.query(
    """
MATCH (n)
DETACH DELETE n
"""
)
graph.refresh_schema()
print(graph.schema)

Node properties are the following:

Relationship properties are the following:

The relationships are the following:



In [3]:
import logging
from urllib.parse import urljoin
import requests
from bs4 import BeautifulSoup

from loader.radiopaedia import RadioWebLoader

search_query = "atelectasis"
loader = RadioWebLoader(
    search_query,
    only_first=True,
)
raw_documents = loader.load()
text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
documents = text_splitter.split_documents(
    raw_documents
)  # should I only take the first? which is the most relevant?

In [4]:
# from langchain.document_loaders import DiffbotLoader

# raw_documents = DiffbotLoader(
#     urls=[all_address[0]],
#     api_token=DIFFBOT_API_TOKEN,
# ).load()
# text_splitter = TokenTextSplitter(chunk_size=2048, chunk_overlap=24)
# documents = text_splitter.split_documents(
#     raw_documents
# )  # should I only take the first? which is the most relevant?

In [5]:
from tqdm import tqdm
from aug.graph_doc import (
    get_extraction_chain,
    data_to_graph_doc,
    chain_run,
    add_graph_documents,
)
from langchain.chat_models import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0)

allowed_nodes = None
allowed_rels = None
# allowed_nodes = ["Symptom", "Disease"]
# allowed_rels = ["CAN_CAUSE", "DESCRIBE", "HAS"]

extract_chain = get_extraction_chain(llm, allowed_nodes, allowed_rels)
gds = []

for d in tqdm(documents, total=len(documents)):
    data = chain_run(extract_chain, d.page_content)
    # data = extract_chain.run(d.page_content)
    # graph_document = GraphDocument(
    #     nodes=[map_to_base_node(node) for node in data.nodes],
    #     relationships=[map_to_base_relationship(rel) for rel in data.rels],
    #     source=d,
    # )
    graph_document = data_to_graph_doc(data, d)
    # add_graph_document(graph, graph_document)
    gds.append(graph_document)

add_graph_documents(graph, gds)

graph.refresh_schema()
print(graph.schema)

c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use langchain_openai.ChatOpenAI instead.
  warn_deprecated(
  0%|          | 0/2 [00:00<?, ?it/s]c:\Users\mike8\anaconda3\envs\pytorch\lib\site-packages\langchain_core\_api\deprecation.py:115: LangChainDeprecationWarning: The function `run` was deprecated in LangChain 0.1.0 and will be removed in 0.2.0. Use invoke instead.
  warn_deprecated(
100%|██████████| 2/2 [00:19<00:00,  9.95s/it]

Node properties are the following:
Condition {name: STRING, id: STRING},Category {id: STRING, name: STRING},Medicalcondition {summary: STRING, id: STRING, name: STRING},Medical imaging {name: STRING, id: STRING},Body part {id: STRING, name: STRING},Finding {name: STRING, id: STRING},Presentation {id: STRING, name: STRING},Event {name: STRING, id: STRING},Injury {id: STRING, name: STRING}
Relationship properties are the following:

The relationships are the following:



In [6]:
from langchain.chains import GraphCypherQAChain
chain = GraphCypherQAChain.from_llm(
    graph=graph,
    cypher_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    qa_llm=ChatOpenAI(temperature=0, model="gpt-3.5-turbo-16k"),
    validate_cypher=True,  # Validate relationship directions
    verbose=True,
)

In [7]:
lesion = "Atelectasis"
questions = [
    f"What is {lesion}?",
    f"What are the symptoms associated with {lesion}?",
    f"What can cause {lesion}?",
    f"What are the patient’s symptoms that are relevant for {lesion}?",
    f"What are the relevant clinical signs for the etiological diagnosis of {lesion}?",
    f"What are the relevant laboratory data for the etiological diagnosis of {lesion}?",
    f"What are the relevant clinical characteristics for the etiological diagnosis of {lesion}",
    f"What are the patient’s personal relevant history for the etiological diagnosis of {lesion}",
]

In [9]:
res_dict = {}
for q in questions:
    try:
        res = chain.run(q)
        res_dict[q] = res.strip()
    except:
        res_dict[q] = "Generated Cypher Statement is not valid"
# res = chain.run("What is Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (mc:Medicalcondition {name: "Atelectasis"})
RETURN mc
Full Context:
[{'mc': {'summary': 'Atelectasis describes small areas of collapsed lung. Atelectasis and collapse both describe the same pathophysiology, though atelectasis tends to be used to describe small areas of lung that are not fully expanded, whereas collapse tends to be used to describe larger more confluent areas.', 'name': 'Atelectasis', 'id': 'Atelectasis'}}]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> Finished chain.


> Entering new GraphCypherQAChain chain...
Generated Cypher:

Full Context:
[]

> F

In [10]:
for q in questions:
    print(res_dict[q])

Atelectasis is a condition that describes small areas of collapsed lung. It is often used to describe areas of the lung that are not fully expanded. Atelectasis and collapse are essentially the same in terms of their underlying pathophysiology, but atelectasis is typically used to refer to smaller areas while collapse is used for larger, more confluent areas.
I'm sorry, but I don't have the information to answer your question. It would be best to consult a medical professional for accurate information on the symptoms associated with Atelectasis.
There are several factors that can cause atelectasis. Some common causes include blockage of the airways due to mucus, a foreign object, or a tumor, as well as lung diseases such as chronic obstructive pulmonary disease (COPD) or pneumonia. Additionally, certain medical procedures, such as anesthesia or prolonged bed rest, can also contribute to the development of atelectasis. It is important to consult with a healthcare professional for a prop

In [ ]:
raise StopIteration()

In [ ]:
chain.run("What is Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medicalterm {name: 'Atelectasis'}) RETURN m
Full Context:
[{'m': {'name': 'Atelectasis', 'id': 'Atelectasis'}}]

> Finished chain.


'Atelectasis is a medical condition characterized by the collapse or closure of a lung or a part of it.'

In [ ]:
chain.run("What are the symptoms of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medicalterm {name: 'Atelectasis'})-[:DESCRIBES]->(s:Medicalterm)
RETURN s.name AS Symptom
Full Context:
[{'Symptom': 'Collapse'}]

> Finished chain.


'The symptom of Atelectasis is collapse.'

In [ ]:
chain.run("What can cause Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medicalterm {name: 'Atelectasis'})-[:DESCRIBES]->(m2:Medicalterm)
RETURN m2.name
Full Context:
[{'m2.name': 'Collapse'}]

> Finished chain.


'A possible cause of Atelectasis is a condition called Collapse.'

In [ ]:
chain.run("What clinical features radiologists need for diagnosing Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medicalterm {name: 'Atelectasis'})-[:DESCRIBES]->(m2:Medicalterm)-[:HAS]->(t:Topic)-[:HAS]->(i:Imagingtechnique)
RETURN i.name
Full Context:
[]

> Finished chain.


'To diagnose Atelectasis, radiologists typically look for certain clinical features. These may include signs such as lung collapse, airway obstruction, and reduced lung volume. Additionally, they may also consider the presence of symptoms such as shortness of breath, coughing, and decreased breath sounds. By evaluating these clinical features, radiologists can help in diagnosing Atelectasis.'

In [ ]:
chain.run("what are radiographic features of Atelectasis?")



> Entering new GraphCypherQAChain chain...
Generated Cypher:
MATCH (m:Medicalterm {name: 'Atelectasis'})-[:DESCRIBES]->(m2:Medicalterm)-[:HAS]->(t:Topic)-[:HAS]->(i:Imagingtechnique)
RETURN i.name
Full Context:
[]

> Finished chain.


"I'm sorry, but I don't have the information to answer your question about the radiographic features of Atelectasis."